
# Event-Driven Backtesting System Demonstration

This notebook demonstrates how to use an event-driven backtesting system with provided example data. 
The following sections will cover:

1. Loading the required data files (event signals, stock prices, TAIEX index data).
2. Initializing and running the backtesting system.
3. Calculating and visualizing the backtesting results.

---


In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

sys.path.extend(['../', '../../'])
from src.backtest import EventBacktester
from src.metrics import MetricsCalculator
from src.utils import load_data, trade_conditions

# File paths (adjust these paths if necessary)
events_path = r'C:\Users\User\Desktop\event-trading\notebooks\events.csv'
prices_path = r'C:\Users\User\Desktop\event-trading\notebooks\prices.csv'
taiex_path = r'C:\Users\User\Desktop\event-trading\notebooks\TAIEX.csv'


### Load Data

In [2]:

# Load the events, prices, and TAIEX data
events_df = pd.read_csv(events_path)
prices_df = pd.read_csv(prices_path)
taiex_df = pd.read_csv(taiex_path)

# Convert datetime columns to actual datetime type and set as index
events_df['datetime'] = pd.to_datetime(events_df['datetime'])
prices_df['datetime'] = pd.to_datetime(prices_df['datetime'])
taiex_df['datetime'] = pd.to_datetime(taiex_df['datetime'])

events_df.set_index('datetime', inplace=True)
prices_df.set_index('datetime', inplace=True)
taiex_df.set_index('datetime', inplace=True)

# Only keep the closing prices as a Series in TAIEX data
taiex_returns = taiex_df['指數收盤價'].pct_change().fillna(0)


In [3]:
events_df.head()

,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
2013-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03,28.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-04,26.0,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-07,31.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-08,28.0,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
prices_df.head()

,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
2013-01-02,39.00,37.45,13.85,16.15,6.71,10.70,13.95,38.55,20.25,26.90,...,16.55,62.7,19.05,20.5,19.2,42.3,29.10,13.50,17.25,12.70
2013-01-03,38.85,37.50,13.90,16.35,6.80,10.65,14.05,38.50,20.40,26.80,...,16.60,67.0,18.95,20.2,19.2,42.2,29.25,13.35,17.30,12.85
2013-01-04,39.00,37.20,13.80,16.65,6.92,10.65,13.80,38.35,20.25,26.90,...,16.55,66.1,19.10,20.3,19.4,41.6,29.25,13.15,17.30,13.00
2013-01-07,38.90,37.00,13.90,17.00,7.02,10.70,14.00,39.60,20.65,27.15,...,17.20,65.6,19.50,20.0,19.5,41.7,29.40,13.45,17.20,12.85
2013-01-08,38.50,36.85,13.95,17.10,6.98,10.55,14.05,39.15,20.40,27.20,...,17.50,64.5,19.25,19.8,19.3,41.7,28.95,13.25,17.10,12.80


In [5]:
taiex_df.head()

,指數收盤價
datetime,
2013-01-02,7779.22
2013-01-03,7836.84
2013-01-04,7805.99
2013-01-07,7755.09
2013-01-08,7721.66


### Initialize and Apply Trade Conditions

In [6]:
def trade_conditions2(row):
    new_row = row.copy()
    for symbol in row.index:
        if pd.notna(row[symbol]):
            new_row[symbol] = "buy"
        # 如果是 NaN，就保持原值（不改動）
    return new_row

In [7]:

# Initialize the backtesting system
backtester = EventBacktester(events_df=events_df, prices_df=prices_df, trade_conditions=trade_conditions2)

# Apply trade conditions to convert events to buy/sell signals
backtester.apply_trade_conditions()
backtester.events_df.head()  # Display the updated events_df with signals


C:\Users\User\AppData\Local\Temp\ipykernel_23368\1290364975.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'buy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_row[symbol] = "buy"


,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
2013-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03,buy,buy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,buy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-04,buy,buy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,buy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-07,buy,buy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,buy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-08,buy,buy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,buy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Execute Trades

In [8]:

# Execute the trades based on signals
backtester.execute_trades()

# Display the trades executed
pd.DataFrame(backtester.trades).head()


,date,stock_code,action,amount,price,fee,pnl
0,2013-01-03,1101,buy,1,38.85,0.033217,-0.033217
1,2013-01-03,1102,buy,1,37.50,0.032063,-0.032063
2,2013-01-03,1216,buy,1,53.90,0.046084,-0.046084
3,2013-01-03,1227,buy,1,82.00,0.070110,-0.070110
4,2013-01-03,1301,buy,1,79.80,0.068229,-0.068229


### Calculate Metrics

In [9]:

# Initialize metrics calculator with trades and TAIEX returns
metrics_calculator = MetricsCalculator(pd.DataFrame(backtester.trades))
metrics_calculator.taiex_returns = taiex_returns

# Calculate cumulative returns and other metrics
cumulative_returns = metrics_calculator.calculate_cumulative_returns()
metrics = metrics_calculator.calculate_all_metrics()

# Display metrics
metrics


c:\Users\User\Desktop\event-trading\notebooks\..\src\metrics.py:42: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  taiex_cumulative = taiex_cumulative.reindex(self.trades_df['date'].drop_duplicates()).fillna(method='ffill')
c:\Users\User\Desktop\event-trading\notebooks\..\src\metrics.py:22: RuntimeWarning: invalid value encountered in scalar power
  return (1 + final_value) ** (365 / total_days) - 1


{'PnL': -97.41288599999999,
 'Sharpe Ratio': -181.06023298279968,
 'CAGR': nan,
 'MDD': -0.0}

### Plot Cumulative Returns

In [10]:

# Plot the cumulative returns
metrics_calculator.plot_cumulative_returns()
plt.show()


TypeError: MetricsCalculator.plot_cumulative_returns() missing 1 required positional argument: 'cumulative_returns'